In [14]:
import requests
import csv


class WaterQualityAPI:
    def __init__(self, base_url):
        self.base_url = base_url

    def get_data(self, data_type):

        url = f"{self.base_url}/{data_type}.json"
        try:
            response = requests.get(url)
            response.raise_for_status()  
            return response.json()  
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {data_type}: {e}")
            return None

    def save_to_csv(self, data, filename):
        if data:

            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=data[0].keys())
                writer.writeheader()  
                writer.writerows(data)  
            print(f"Data saved to {filename}")
        else:
            print("No data to save.")


def limit_rows(data, limit):
    """Limit the number of rows to avoid memory errors."""
    return data[:limit] if len(data) > limit else data


def merge_data(sensors, reports, ph, temperature, turbidity):
    merged = []

    for report in reports:

        if 'ReservoirID' not in report:
            print(f"Skipping report {report['ReportID']} due to missing ReservoirID")
            continue


        sensor_data = next(
            (s for s in sensors if s['ReservoirID'] == report['ReservoirID']), None)


        ph_data = next(
            (p for p in ph if p['ReservoirID'] == report['ReservoirID']), None)


        temperature_data = next(
            (t for t in temperature if t['ReservoirID'] == report['ReservoirID']), None)


        turbidity_data = next(
            (t for t in turbidity if t['ReservoirID'] == report['ReservoirID']), None)


        if sensor_data:

            merged_record = {**report}


            if sensor_data:
                merged_record.update(sensor_data)


            if ph_data:
                merged_record.update(ph_data)


            if temperature_data:
                merged_record.update(temperature_data)


            if turbidity_data:
                merged_record.update(turbidity_data)

            merged.append(merged_record)

    return merged


def write_to_csv(merged_data, filename):
    """Write the merged data into a CSV file."""
    if merged_data:

        header = merged_data[0].keys()
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=header)
            writer.writeheader()
            writer.writerows(merged_data)  
        print(f"CSV file '{filename}' written successfully.")
    else:
        print("No data to write to CSV.")


def fetch_and_merge_data():
    api = WaterQualityAPI("http://localhost:8081")


    data_types = ['sensors', 'reports', 'ph', 'temperature', 'turbidity']

    data = {}


    for data_type in data_types:
        print(f"Fetching data for {data_type}...")
        data[data_type] = api.get_data(data_type)
        print(f"First record in {data_type} data: {data[data_type][0] if data[data_type] else 'No data'}")


    data['sensors'] = limit_rows(data['sensors'], 50)
    data['reports'] = limit_rows(data['reports'], 50)
    data['ph'] = limit_rows(data['ph'], 20)
    data['temperature'] = limit_rows(data['temperature'], 20)
    data['turbidity'] = limit_rows(data['turbidity'], 20)


    merged_data = merge_data(
        data['sensors'], data['reports'], data['ph'], data['temperature'], data['turbidity'])


    write_to_csv(merged_data, 'merged_web.csv')


if __name__ == "__main__":
    fetch_and_merge_data()

Fetching data for sensors...
First record in sensors data: {'SensorID': 1, 'SensorType': 'Temperature', 'Location': 'Reservoir_3', 'ReservoirID': 3, 'InstallationDate': '2022-11-18T00:00:00', 'Status': 0}
Fetching data for reports...
First record in reports data: {'ReportID': 1, 'Validation': 0, 'Time': '2024-11-23T00:00:00', 'ReservoirID': 1, 'Temperature': 10.67, 'Turbidity': 2.56, 'pH': 7.88, 'ReportURL': 'report_1.pdf'}
Fetching data for ph...
First record in ph data: {'pHDataID': 1, 'pH': 7.7, 'Time': '2024-10-02T00:00:00', 'SensorID': 75, 'ReservoirID': 4, 'Report_ReportID': 695}
Fetching data for temperature...
First record in temperature data: {'TempRecordDataID': 1, 'Temperature': 10.25, 'Time': '2024-10-28T00:00:00', 'SensorID': 95, 'ReservoirID': 2, 'Report_ReportID': 9}
Fetching data for turbidity...
First record in turbidity data: {'TurbidityDataID': 1, 'Turbidity': 2.67, 'Time': '2024-02-07T00:00:00', 'SensorID': 8, 'ReservoirID': 4, 'Report_ReportID': 152}
CSV file 'merg